In [89]:
# imports
import altair as alt
import pandas as pd
import requests
# source: Deloitte Consulting - Annual review of football finance 2025
# https://www.deloitte.com/content/dam/assets-zone2/uk/en/docs/services/consulting/2025/deloitte-annual-review-of-football-finance-2025.pdf

In [90]:
# Import pound currency formatting
gb_format = 'https://raw.githubusercontent.com/d3/d3-format/refs/heads/main/locale/en-GB.json'

response = requests.get(gb_format)
gb_format = response.json()

alt.renderers.set_embed_options(formatLocale=gb_format)

RendererRegistry.enable('default')

In [65]:
# Creating Premier League revenue and wage cost table based on Chart 8
prem_teams =  pd.DataFrame({
    'club': ['ManC', 'ManU', 'Liverpool', 'Arsenal', 'TotHot', 'Chelsea', 'Newcastle', 'AstonVilla', 'WestHam', 'B&HA', 'CrystalPalace', 'Everton', 'Fulham', 'WW', 'Nottingham', 'Brentford', 'Bournmouth', 'Sheffield', 'Burnley', 'Luton'],
    'total': [719, 662, 614, 613, 518, 468, 320, 272, 271, 223, 189, 187, 181, 178, 174, 169, 161, 138, 134, 132],
    'Wage Cost': [413 , 365 , 386 , 328 , 222 , 338 , 219 , 252 , 161 , 146 , 134 , 157 , 157 , 142 , 167 , 114 , 136 , 65 , 92 , 57]
    
})

In [66]:
# Add column to calculate revenue
prem_teams['Revenue'] = prem_teams['total'] - prem_teams['Wage Cost']

In [67]:
prem_revcost = pd.DataFrame({
    'league': ['Premier', 'Premier'],
    'rev/cost': ['Revenue','Wage Cost'],
    'amt':[prem_teams['Revenue'].sum(),prem_teams['Wage Cost'].sum()]
})

In [69]:
# Creating revenue and wage cost table for lower leagues based on Chart 11
lower_leagues_revcost = pd.DataFrame({
    'league': ['Championship', 'Championship' , 'L1', 'L1', 'L2', 'L2'],
    'rev/cost': ['Revenue', 'Wage Cost', 'Revenue', 'Wage Cost', 'Revenue', 'Wage Cost'],
    'amt': [66, 892, 10, 208, 42, 118]
})

In [70]:
# Create empty dataframe to hold chart
chart_table = pd.DataFrame()
# Concat prem and lower league revenue/wcost charts
chart_table = pd.concat([prem_revcost, lower_leagues_revcost], axis=0)

In [91]:
# Create grouped bar chart - WITH FACETS
chart_table['league'] = pd.Categorical(chart_table['league'], ['Premier','Championship','L1','L2'], ordered=True)

alt.Chart(chart_table).mark_bar().encode(
    x='rev/cost',
    y=alt.Y('amt', title='(£) in millions', axis=alt.Axis(titleX=-70)),
    column=alt.Column('league', title=None),
    color='rev/cost',
    tooltip=alt.Tooltip('amt', format='$,.0f')
).properties(title='Total revenue and wage cost by league',width=100)

alt.Chart(...)

In [194]:
# Create grouped bar chart - WITH OFFSET (allows for text labels)
chart_table['league'] = pd.Categorical(chart_table['league'], ['Premier','Championship','L1','L2'], ordered=True)

base = alt.Chart(chart_table).encode(
    x= alt.X('league:N', sort=['Premier','Championship','L1','L2']),
    y= alt.Y('amt:Q', title=None,  axis=alt.Axis(titleX=-70,
                                                             labelExpr='"£" + datum.value / 1000 + " bn"'
                                                             )),
    xOffset = alt.XOffset('rev/cost:N', scale=alt.Scale(paddingOuter=0)),
    color= alt.Color('rev/cost:N', legend= alt.Legend(orient='top'))
).properties(
    title={'text': 'Football Finances: Revenue and wage cost by league', 'subtitle': 'Source: Deloitte Consulting - Annual review of football finance 2025'}
    )

layers= alt.layer(
    base.mark_bar(),
    base.mark_text(align='center', text=alt.expr("'£' + format(datum.amt/1000, '.1f') + ' bn'"), baseline='top', dx= 0, dy = -12).encode(color=alt.value('black'))

)

In [195]:
layers.display()

alt.LayerChart(...)

In [177]:
from ecostyles import EcoStyles
# Create styles instance
styles = EcoStyles()
# Register and enable a theme
styles.register_and_enable_theme(theme_name="article")  # or "article"


In [196]:
# Save to png
layers.save('football_finances_1.png', scale_factor=2)

In [197]:
# Save to json
layers.save('football_finances_1.json', scale_factor=2)

In [8]:
# styles.save(layers, 'charts', 'football_finances_1', width=500, height=400, svg=True)

In [152]:
# Use helper methods
#styles.add_source(layers, 'Deloitte Consulting - Annual review of football finance 2025')
# styles.save(layers, '/Users/sambickel-barlow/Desktop/Github/RADataHub/ChartOfTheDay/sports/charts', 'football_finances_1')